In [3]:
#Library untuk manipulasi dataframe
import pandas as pd
#Fungsi matematika, kita hanya perlu fungsi sqrt jadi mari kita impor saja
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('anime_with_synopsis.csv')
df.head(5)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [5]:
jumlah_kosong_per_kolom = df.isna().sum()
print(jumlah_kosong_per_kolom)

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64


In [6]:
# Menghapus baris yang mengandung data kosong
df_cleaned = df.dropna()

# Menampilkan DataFrame setelah data kosong dihapus
print(df_cleaned)

       MAL_ID                             Name    Score  \
0           1                     Cowboy Bebop     8.78   
1           5  Cowboy Bebop: Tengoku no Tobira     8.39   
2           6                           Trigun     8.24   
3           7               Witch Hunter Robin     7.27   
4           8                   Bouken Ou Beet     6.98   
...       ...                              ...      ...   
16209   48481  Daomu Biji Zhi Qinling Shen Shu  Unknown   
16210   48483                     Mieruko-chan  Unknown   
16211   48488  Higurashi no Naku Koro ni Sotsu  Unknown   
16212   48491      Yama no Susume: Next Summit  Unknown   
16213   48492                    Scarlet Nexus  Unknown   

                                                  Genres  \
0        Action, Adventure, Comedy, Drama, Sci-Fi, Space   
1                  Action, Drama, Mystery, Sci-Fi, Space   
2      Action, Sci-Fi, Adventure, Comedy, Drama, Shounen   
3      Action, Mystery, Police, Supernatural, Drama

In [7]:
df_cleaned.isna().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    0
dtype: int64

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi objek TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan ekstraksi fitur menggunakan TF-IDF
features = vectorizer.fit_transform(df.Genres)

# Menampilkan hasil ekstraksi fitur
print(features.toarray())
print(features.shape)

[[0.31036127 0.34445181 0.         ... 0.         0.         0.        ]
 [0.29720298 0.         0.         ... 0.         0.         0.        ]
 [0.34219477 0.37978195 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.42348562 0.         ... 0.         0.         0.        ]
 [0.68106111 0.         0.         ... 0.         0.         0.        ]]
(16214, 46)


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity antara dua vektor fitur
similarity_matrix = cosine_similarity(features)

# Menampilkan hasil cosine similarity
print(similarity_matrix)
print(similarity_matrix.shape)

[[1.         0.78205726 0.74170397 ... 0.         0.22538012 0.21137499]
 [0.78205726 1.         0.51670652 ... 0.18244967 0.         0.20241339]
 [0.74170397 0.51670652 1.         ... 0.         0.24849717 0.23305555]
 ...
 [0.         0.18244967 0.         ... 1.         0.         0.        ]
 [0.22538012 0.         0.24849717 ... 0.         1.         0.        ]
 [0.21137499 0.20241339 0.23305555 ... 0.         0.         1.        ]]
(16214, 16214)


In [13]:
def anime_recomendation(title, cos_sim, data):
    index = data[data['Name'] == title].index[0]
    similar = list(enumerate(cos_sim[index]))
    similar = sorted(similar, key=lambda x: x[1], reverse=True)
    
    # return top 5 of anime index include the title that inputed
    return [i[0] for i in similar[0:6]]
    

In [14]:
df_recomendation = df[['Name', 'Genres']]

In [16]:
title = 'Paprika'

print(f"Top 5 Recomendation for Anime titled : {title}")
df_recomendation.iloc[anime_recomendation(title, similarity_matrix, df)]


Top 5 Recomendation for Anime titled : Paprika


,Name,Genres
1691,Paprika,"Dementia, Fantasy, Horror, Mystery, Psychologi..."
824,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
15171,Higurashi no Naku Koro ni Gou,"Mystery, Dementia, Horror, Psychological, Supe..."
16211,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe..."
13265,Yakusoku no Neverland,"Sci-Fi, Mystery, Horror, Psychological, Thrill..."
6598,Higurashi no Naku Koro ni Kaku: Outbreak,"Mystery, Horror, Psychological, Thriller"


In [18]:
input_title = input("Masukkan judul anime: ")

# Mencari indeks anime yang sesuai dengan input
input_index = df[df["Name"].str.lower() == input_title.lower()].index[0]

# Menghitung similarity matrix
similarity_matrix = cosine_similarity(features)

# Mendapatkan nilai kemiripan untuk anime yang sesuai dengan input
similarities = similarity_matrix[input_index]

# Mengurutkan indeks berdasarkan nilai kemiripan (dari yang tertinggi)
sorted_indices = similarities.argsort()[::-1]

# Mengambil 5 judul anime dengan nilai kemiripan tertinggi (selain input)
top_similar_anime_indices = sorted_indices[1:6]
top_similar_anime_titles = df.loc[top_similar_anime_indices, "Name"]

# Menampilkan judul anime dengan nilai kemiripan tertinggi
print("Judul anime dengan nilai kemiripan tertinggi:")
for title in top_similar_anime_titles:
    print(title)

Judul anime dengan nilai kemiripan tertinggi:
Higurashi no Naku Koro ni Gou
Higurashi no Naku Koro ni Sotsu
Higurashi no Naku Koro ni
Yakusoku no Neverland
Higurashi no Naku Koro ni Kaku: Outbreak
